IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M5\FAULT_M5_10


In [3]:
guasto=2

VOLO m5 FAULT 10%

In [4]:
rcou_m5_fault10 = pd.read_csv("RCOU.csv")
rcou_m5_fault10 = rcou_m5_fault10.astype("float64")
rcou_m5_fault10 = rcou_m5_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m5_fault10 = rcou_m5_fault10[((rcou_m5_fault10['C9'] >= potenza) & (rcou_m5_fault10['C10'] >= potenza) & (rcou_m5_fault10['C11'] >= potenza) & (rcou_m5_fault10['C12'] >= potenza) & (rcou_m5_fault10['C13'] >= potenza) & (rcou_m5_fault10['C14']>= potenza))]
rcou_m5_fault10=rcou_m5_fault10.reset_index(drop=True)
display(rcou_m5_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,72720687.0,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0
1,72731556.0,1415.0,1540.0,1494.0,1466.0,1428.0,1529.0
2,72741603.0,1472.0,1506.0,1414.0,1556.0,1393.0,1572.0
3,72752252.0,1524.0,1471.0,1426.0,1562.0,1454.0,1539.0
4,72762572.0,1495.0,1518.0,1476.0,1535.0,1461.0,1549.0
...,...,...,...,...,...,...,...
7390,166795551.0,1497.0,1409.0,1482.0,1427.0,1398.0,1507.0
7391,166805911.0,1488.0,1417.0,1464.0,1443.0,1369.0,1527.0
7392,166817138.0,1469.0,1437.0,1481.0,1424.0,1367.0,1528.0
7393,166827779.0,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0


In [5]:
min=rcou_m5_fault10['TimeUS'][0]
max=rcou_m5_fault10['TimeUS'][len(rcou_m5_fault10)-1]
print(max)
print(min)

166914844.0
72720687.0


In [6]:
xkf1_m5_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m5_fault10 = xkf1_m5_fault10.astype("float64")
xkf1_m5_fault10 = xkf1_m5_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_fault10 = xkf1_m5_fault10[((xkf1_m5_fault10['TimeUS'] >= min) & (xkf1_m5_fault10['TimeUS'] <= max	))]
xkf1_m5_fault10 = xkf1_m5_fault10.reset_index(drop=True)
print(xkf1_m5_fault10)

           TimeUS  Roll  Pitch   Yaw
0      72729675.0 -0.65   0.55  2.33
1      72739678.0 -0.66   0.65  2.33
2      72750119.0 -0.61   0.61  2.33
3      72760283.0 -0.65   0.60  2.34
4      72770308.0 -0.62   0.57  2.37
...           ...   ...    ...   ...
8533  166869222.0  1.82  -2.80  4.06
8534  166879639.0  1.70  -2.83  4.07
8535  166890212.0  1.68  -2.82  4.08
8536  166900589.0  1.69  -2.85  4.07
8537  166912734.0  1.77  -2.79  4.08

[8538 rows x 4 columns]


In [7]:
att_m5_fault10 = pd.read_csv("ATT.csv")
att_m5_fault10 = att_m5_fault10.astype("float64")
att_m5_fault10 = att_m5_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_fault10 = att_m5_fault10[((att_m5_fault10['TimeUS'] >= min) & (att_m5_fault10['TimeUS'] <= max	))]
att_m5_fault10=att_m5_fault10.reset_index(drop=True)
print(att_m5_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0      72730714.0    -0.21 -0.65     -0.06   0.55    2.27  2.33
1      72740788.0    -0.22 -0.66     -0.06   0.65    2.27  2.33
2      72751532.0    -0.23 -0.61     -0.07   0.61    2.27  2.33
3      72761836.0    -0.22 -0.65     -0.07   0.60    2.27  2.34
4      72772125.0    -0.21 -0.62     -0.09   0.57    2.27  2.37
...           ...      ...   ...       ...    ...     ...   ...
8534  166870527.0     1.97  1.82      0.13  -2.80    5.21  4.06
8535  166881106.0     1.95  1.70      0.12  -2.83    5.21  4.07
8536  166891684.0     1.92  1.68      0.11  -2.82    5.21  4.08
8537  166902181.0     1.88  1.69      0.11  -2.85    5.21  4.07
8538  166914063.0     1.84  1.77      0.11  -2.79    5.21  4.08

[8539 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m5_fault10)>len(xkf1_m5_fault10):
    to_add=att_m5_fault10[len(xkf1_m5_fault10):]
    att_m5_fault10=att_m5_fault10[:len(xkf1_m5_fault10)]

if len(xkf1_m5_fault10)>len(att_m5_fault10):
    to_add=xkf1_m5_fault10[len(att_m5_fault10):]

for idx,i in enumerate(att_m5_fault10['Roll']):
    if(xkf1_m5_fault10['Roll'][idx] != i):
        att_m5_fault10['Roll'][idx] = ((att_m5_fault10['Roll'][idx] + xkf1_m5_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m5_fault10['Pitch']):
    if(xkf1_m5_fault10['Pitch'][idx] != i):
        att_m5_fault10['Pitch'][idx] = ((att_m5_fault10['Pitch'][idx] + xkf1_m5_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m5_fault10['Yaw']):
    if(xkf1_m5_fault10['Yaw'][idx] != i):
        att_m5_fault10['Yaw'][idx] = ((att_m5_fault10['Yaw'][idx] + xkf1_m5_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m5_fault10=pd.concat([att_m5_fault10,to_add])

print(att_m5_fault10)

           TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw
0      72730714.0    -0.21 -0.650     -0.06  0.550    2.27  2.330
1      72740788.0    -0.22 -0.660     -0.06  0.650    2.27  2.330
2      72751532.0    -0.23 -0.610     -0.07  0.610    2.27  2.330
3      72761836.0    -0.22 -0.650     -0.07  0.600    2.27  2.340
4      72772125.0    -0.21 -0.620     -0.09  0.570    2.27  2.370
...           ...      ...    ...       ...    ...     ...    ...
8534  166870527.0     1.97  1.760      0.13 -2.815    5.21  4.065
8535  166881106.0     1.95  1.690      0.12 -2.825    5.21  4.075
8536  166891684.0     1.92  1.685      0.11 -2.835    5.21  4.075
8537  166902181.0     1.88  1.730      0.11 -2.820    5.21  4.075
8538  166914063.0     1.84  1.770      0.11 -2.790    5.21  4.080

[8539 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m5_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m5_fault10 = esc_0_m5_fault10.astype("float64")
esc_0_m5_fault10 = esc_0_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_fault10 = esc_0_m5_fault10[((esc_0_m5_fault10['TimeUS'] >= min) & (esc_0_m5_fault10['TimeUS'] <= max))]
esc_0_m5_fault10=esc_0_m5_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m5_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m5_fault10 = esc_1_m5_fault10.astype("float64")
esc_1_m5_fault10 = esc_1_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m5_fault10 = esc_1_m5_fault10[((esc_1_m5_fault10['TimeUS'] >= min) & (esc_1_m5_fault10['TimeUS'] <= max))]
esc_1_m5_fault10=esc_1_m5_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m5_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m5_fault10 = esc_2_m5_fault10.astype("float64")
esc_2_m5_fault10 = esc_2_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m5_fault10 = esc_2_m5_fault10[((esc_2_m5_fault10['TimeUS'] >= min) & (esc_2_m5_fault10['TimeUS'] <= max))]
esc_2_m5_fault10=esc_2_m5_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m5_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m5_fault10 = esc_3_m5_fault10.astype("float64")
esc_3_m5_fault10 = esc_3_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m5_fault10 = esc_3_m5_fault10[((esc_3_m5_fault10['TimeUS'] >= min) & (esc_3_m5_fault10['TimeUS'] <= max))]
esc_3_m5_fault10=esc_3_m5_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m5_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m5_fault10 = esc_4_m5_fault10.astype("float64")
esc_4_m5_fault10 = esc_4_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m5_fault10 = esc_4_m5_fault10[((esc_4_m5_fault10['TimeUS'] >= min) & (esc_4_m5_fault10['TimeUS'] <= max))]
esc_4_m5_fault10=esc_4_m5_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m5_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m5_fault10 = esc_5_m5_fault10.astype("float64")
esc_5_m5_fault10 = esc_5_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m5_fault10 = esc_5_m5_fault10[((esc_5_m5_fault10['TimeUS'] >= min) & (esc_5_m5_fault10['TimeUS'] <= max))]
esc_5_m5_fault10=esc_5_m5_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m5_fault10 = pd.read_csv("ESC_0.csv")\nesc_0_m5_fault10 = esc_0_m5_fault10.astype("float64")\nesc_0_m5_fault10 = esc_0_m5_fault10.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m5_fault10 = esc_0_m5_fault10[((esc_0_m5_fault10[\'TimeUS\'] >= min) & (esc_0_m5_fault10[\'TimeUS\'] <= max))]\nesc_0_m5_fault10=esc_0_m5_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m5_fault10 = pd.read_csv("ESC_1.csv")\nesc_1_m5_fault10 = esc_1_m5_fault10.astype("float64")\nesc_1_m5_fault10 = esc_1_m5_fault10.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m5_fault10 = esc_1_m5_fault10[((esc_1_m5_fault10[\'TimeUS\'] >= min) & (esc_1_m5_fault10[\'TimeUS\'] <= max))]\nesc_1_m5_fault10=esc_1_m5_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\n

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m5_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m5_fault10 = imu_0_m5_fault10.astype("float64")
imu_0_m5_fault10 = imu_0_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m5_fault10 = imu_0_m5_fault10[((imu_0_m5_fault10['TimeUS'] >= min) & (imu_0_m5_fault10['TimeUS'] <= max))]

imu_1_m5_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m5_fault10 = imu_1_m5_fault10.astype("float64")
imu_1_m5_fault10 = imu_1_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m5_fault10 = imu_1_m5_fault10[((imu_1_m5_fault10['TimeUS'] >= min) & (imu_1_m5_fault10['TimeUS'] <= max))]

imu_2_m5_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m5_fault10 = imu_2_m5_fault10.astype("float64")
imu_2_m5_fault10 = imu_2_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_fault10 = imu_2_m5_fault10[((imu_2_m5_fault10['TimeUS'] >= min) & (imu_2_m5_fault10['TimeUS'] <= max))]

imu_m5_fault10 = pd.concat((imu_0_m5_fault10, imu_1_m5_fault10, imu_2_m5_fault10))
imu_m5_fault10=imu_m5_fault10.groupby(imu_m5_fault10.TimeUS, as_index=False).mean()

display(imu_m5_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,72725912.0,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562
1,72728103.0,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303
2,72730554.0,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179
3,72733710.0,0.119616,-0.211923,-0.026932,0.110713,0.545332,-9.907626
4,72735559.0,0.112437,-0.099032,-0.026028,-0.145339,0.406057,-10.143177
...,...,...,...,...,...,...,...
32433,166897233.0,0.031076,0.046551,0.022947,-0.416947,0.473361,-9.839571
32434,166899201.0,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236
32435,166902063.0,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467
32436,166904726.0,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m5_fault10 = pd.merge_ordered(imu_m5_fault10,att_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m5_fault10 = pd.merge_ordered(m5_fault10, esc_0_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_1_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_2_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_3_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_4_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_5_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")'''
m5_fault10 = pd.merge_ordered(m5_fault10, rcou_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m5_fault10["TimeUS"] = m5_fault10["TimeUS"] - m5_fault10.iloc[0]["TimeUS"]
m5_fault10["TimeUS"] = m5_fault10["TimeUS"].astype(int)
m5_fault10["Guasto"] = guasto

In [13]:
last_cell = m5_fault10.iloc[-1, m5_fault10.columns.get_loc('TimeUS')]
print(last_cell)

94194157


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
32965  94181005
32966  94183862
32967  94186719
32968  94189576
32969  94192433

[32970 rows x 1 columns]


In [16]:
m5_fault10_final = pd.merge_ordered(m5_fault10,df)
m5_fault10_final=m5_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m5_fault10_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
1          2857  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
2          5225  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
3          5714  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
4          7416 -0.245029  0.217662  0.038900 -0.212036  0.573743  -9.924303   
...         ...       ...       ...       ...       ...       ...        ...   
81320  94189576  0.015743  0.110068  0.012201 -0.670795  0.566046 -10.021941   
81321  94192433  0.015743  0.110068  0.012201 -0.670795  0.566046 -10.021941   
81322  94193154 -0.053250  0.057050 -0.006003 -0.753840  0.213708 -10.067043   
81323  94193376 -0.053250  0.057050 -0.006003 -0.753840  0.213708 -10.067043   
81324  94194157 -0.053250  0.057050 -0.006003 -0.753840  0.213708 -10.067043   

       DesRoll  Roll  DesPitch  Pitch  

In [17]:
m5_fault10_final = m5_fault10_final[m5_fault10_final.TimeUS.isin(to_be)]

In [18]:
m5_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
1,2857,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
3,5714,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
5,8571,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
9,11428,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179,-0.21,-0.650,-0.06,0.550,2.27,2.330,1415.0,1540.0,1494.0,1466.0,1428.0,1529.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81314,94181005,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236,1.92,1.685,0.11,-2.835,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
81317,94183862,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467,1.88,1.730,0.11,-2.820,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
81319,94186719,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,-2.820,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
81320,94189576,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,-2.820,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0


In [19]:
from datetime import timedelta
m5_fault10_final=m5_fault10_final.reset_index(drop=True)
m5_fault10_final['TimeUS'] = pd.to_datetime(m5_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m5_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
1,00:00:00.002857,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
2,00:00:00.005714,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
3,00:00:00.008571,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303,-0.21,-0.650,-0.06,0.550,2.27,2.330,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
4,00:00:00.011428,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179,-0.21,-0.650,-0.06,0.550,2.27,2.330,1415.0,1540.0,1494.0,1466.0,1428.0,1529.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32965,00:01:34.181005,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236,1.92,1.685,0.11,-2.835,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
32966,00:01:34.183862,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467,1.88,1.730,0.11,-2.820,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
32967,00:01:34.186719,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,-2.820,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
32968,00:01:34.189576,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,-2.820,5.21,4.075,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m5_fault10_final)/350)):
        V1=m5_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_13828\1138947296.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_13828\1138947296.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.055093,0.029366,-1.086720,0.179770,0.0,100.0,10.166016,2.788774,-0.010972,0.007536,-0.845377,0.087376,99.0,72.0,2.412127,2.360274,0.145766,0.011018,0.522951,0.179540,62.0,61.0,0.682731,0.570891,-0.245351,0.043550,-0.307544,0.321905,0.0,108.0,19.682915,8.832355,0.493040,0.055320,0.379004,0.546123,95.0,96.0,13.909371,6.453927,-11.552112,0.759988,-0.819016,11.584866,0.0,108.0,154.522153,9.454081,-0.402393,3.260136,-0.955566,1.847368,103.0,101.0,9.808036,4.769784,-0.190171,0.360873,-1.166117,0.629293,0.0,99.0,131.014279,3.526990,6.630085,9.863039,-1.543214,7.334369,93.0,92.0,-2.262845,-2.608520,1656.561254,3188.784094,4.422775,1657.520705,98.0,95.0,2106.757080,834.567064,1511.512821,1968.461978,0.413552,1512.161982,101.0,102.0,1399.105820,1317.673032,1621.108262,3365.976817,2.693952,1622.143146,99.0,101.0,701.370751,668.582372,1552.467236,2177.181066,0.858910,1553.166282,116.0,107.0,972.056474,865.060255,1552.695157,3382.412519,0.818562,1553.780881,99.0,98.0,2714.715398,1459.746680,1619.923077,3598.894066,2.866838,1621.030356,80.0,78.0,2057.160507,555.616639,2
1,1.0,-0.055163,0.029284,-1.080695,0.179565,0.0,94.0,10.428676,2.803751,-0.010751,0.007531,-0.846746,0.087324,98.0,99.0,2.579728,2.183314,0.144820,0.011302,0.578143,0.179563,67.0,66.0,0.693570,0.583309,-0.245919,0.043539,-0.309637,0.322322,0.0,106.0,19.369131,8.346881,0.493596,0.055272,0.379473,0.546582,95.0,96.0,14.664014,6.954487,-11.549535,0.760160,-0.824509,11.582304,0.0,98.0,154.493004,8.949382,-0.412216,3.284811,-0.962911,1.856179,107.0,105.0,9.560706,4.686247,-0.189858,0.359879,-1.160827,0.628413,0.0,107.0,131.270589,3.472707,6.641023,9.877046,-1.545203,7.345214,88.0,65.0,-2.548862,-3.199285,1656.767045,3194.606546,4.409115,1657.728136,93.0,90.0,2099.846289,914.004353,1511.247159,1987.696573,0.377286,1511.902783,99.0,98.0,1296.577257,1272.111459,1621.227273,3361.372701,2.704339,1622.260674,105.0,106.0,688.691372,658.200273,1552.349432,2175.863305,0.844234,1553.048113,116.0,108.0,1091.461829,1008.462655,1552.232955,3447.974068,0.799432,1553.340054,101.0,100.0,2872.390989,1471.211567,1620.284091,3634.517353,2.817385,1621.402087,84.0,82.0,2079.316746,762.188771,2
2,2.0,-0.055297,0.029207,-1.074590,0.179393,0.0,96.0,10.666993,2.793884,-0.010998,0.007531,-0.850666,0.087355,69.0,95.0,3.179913,2.576063,0.143797,0.011640,0.683127,0.179678,61.0,60.0,0.682126,0.553722,-0.246167,0.043437,-0.303710,0.322354,0.0,100.0,19.166273,7.969759,0.493445,0.055123,0.387292,0.546310,103.0,104.0,15.064425,7.175974,-11.547387,0.759629,-0.827317,11.580140,0.0,95.0,154.308011,7.859513,-0.421983,3.309154,-0.970437,1.864899,107.0,80.0,9.319735,4.661844,-0.189547,0.358891,-1.155534,0.627537,0.0,103.0,131.516352,3.423220,6.651898,9.890736,-1.547103,7.355982,93.0,97.0

In [23]:
path_file = path_file.replace(r"M5\FAULT_M5_10", "")
os.chdir(path_file)
df_merged.to_csv('m5_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
